In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [2]:
import yahoo_utils

game_id = 427
players = yahoo_utils.get_players(game_id)
current_schedule = yahoo_utils.get_games_by_week(game_id)
teams = yahoo_utils.get_teams(game_id)
all_matchups = yahoo_utils.get_all_matchups(game_id)

In [411]:
import pandas as pd
preds = pd.read_csv('../data/2324_preds.csv').set_index(['playerId','gameId','playerTeam','opposingTeam','gameDate','week'])
preds = preds.clip(0, np.inf)
y = pd.read_hdf('data/y.h5')
y_g = pd.read_hdf('data/y_g.h5')
y = pd.concat([y,y_g])


def weekify_stats(stats, schedule):
    full_daily_stats = []
    gdates = pd.to_datetime(stats.index.get_level_values('gameDate'), format='%Y%m%d')
    for week, games in schedule.items():
        games_df = pd.DataFrame(games)
        week_dates = pd.date_range(games_df.ts.min(), games_df.ts.max())
        for date in week_dates:
            daily_stats = stats[(gdates == date)].copy()
            daily_stats['date'] = date
            daily_stats['week'] = week
            full_daily_stats.append(daily_stats)
    return pd.concat(full_daily_stats)

daily_preds = weekify_stats(preds, current_schedule)
daily_preds = daily_preds.merge(players[['playerId','name', 'pos', 'player_key']], on='playerId', how='left')

daily_results = weekify_stats(y, current_schedule)
daily_results = daily_results.reset_index().merge(players[['playerId','name', 'pos', 'player_key']], on='playerId', how='left')

In [942]:
from itertools import combinations, product, chain
from ast import literal_eval
import collections

POSITIONS_QUOTA = {
        'D': 4,
        'C': 2,
        'RW': 2,
        'LW': 2,
        'G': 2,
        'WC': 1
}

def check_valid_lineup(lineup, position_lookup, positions_quota=None):
    if positions_quota is None:
        positions_quota = {k: v for k, v in POSITIONS_QUOTA.items()}
    
    positions_map = {k:[] for k in positions_quota}
    multi_positions = []
    for player in lineup:
        if len(position_lookup[player]) == 1:
            pos = position_lookup[player][0]
            if len(positions_map[pos]) == positions_quota[pos]:
                if len(positions_map['WC']) == 0:
                    positions_map['WC'] = [player]
                else:
                    return False
            else:
                positions_map[pos] += [player]     
        else:
            multi_positions.append(player)

    positions_left = [p for p, v in positions_map.items() for i in range(positions_quota[p]-len(v))]
    multi_pos_positions = [position_lookup[p] for p in multi_positions]

    for c in list(product(*multi_pos_positions)):
        if collections.Counter(c) == collections.Counter(positions_left) or len(c) < 13:
            return c
    return False
    

def get_valid_lineups(day_teams, min_players=13):
    position_lookup = day_teams.set_index(['playerId'], drop=False).pos.apply(literal_eval).to_dict()
    n_combs = min(min_players, len(position_lookup))
    lineups = list(combinations(position_lookup.keys(), n_combs))

    
    valid_lineups = [l for l in lineups if check_valid_lineup(l, position_lookup) != False and len(l) > 0]
    if len(valid_lineups) == 0:
        if n_combs > 2:
            return get_valid_lineups(day_teams, min_players=n_combs-1)
    return valid_lineups

def get_lineup_teams(lineups, day_teams):
    teams = []
    for lineup in lineups:
        match_filter = (day_teams.playerId.isin(lineup))
        teams.append(day_teams[match_filter])
    return pd.concat(teams, keys=range(len(teams)), names=['lineup'])


from scipy import stats
from process_data import PRED_COLS

def get_expected_points(exp, opp_exp):
    vals = {}
    
    no_results = exp.isna() | exp == 0
    
    for col in ['g','a','sog','fow','hit','block','pim','ppp', 'win', 'save', 'so']:
        if exp[col] == 0 and opp_exp[col] == 0:
            vals[col] = 0.5
        elif exp[col] == 0:
            vals[col] = stats.poisson.cdf(0, opp_exp[col])
        elif opp_exp[col] == 0:
            vals[col] = 1 - stats.poisson.cdf(0, exp[col])
        else:
            vals[col] = stats.skellam.cdf(-1, opp_exp[col], exp[col]) + 0.5 * stats.skellam.pmf(0, opp_exp[col], exp[col])
    if not no_results['goalsfor'] and not no_results['goalsaga']:
        vals['plusmin'] = stats.skellam.cdf(-1, opp_exp['goalsfor']+exp['goalsaga'], opp_exp['goalsaga']+exp['goalsfor']) \
                        + 0.5 * stats.skellam.pmf(0, opp_exp['goalsfor']+exp['goalsaga'], opp_exp['goalsaga']+exp['goalsfor'])

    if not no_results['icetime'] and not no_results['ga']:
        vals['gaa'] = stats.skellam.cdf(-1, exp['ga']/exp['icetime'], opp_exp['ga']/opp_exp['icetime'])\
                    + 0.5 * stats.skellam.pmf(0, exp['ga']/exp['icetime'], opp_exp['ga']/opp_exp['icetime'])
    return sum(vals.values())

def get_resulted_points(lineup):
    metrics = [m for metrics in PRED_COLS.values() for m in metrics]
    res = lineup[metrics].sum()
    return res

def get_week_result(own_total, opp_total):
    result = 0
    for col in ['g','a','sog','fow','hit','block','pim','ppp', 'win', 'save', 'so']:
        if own_total[col] - opp_total[col] > 0:
            result += 1
        if own_total[col] == opp_total[col]:
            result += 0.5
    plusmin = own_total['goalsfor'] - opp_total['goalsfor'] + opp_total['goalsaga'] - own_total['goalsaga']
    if plusmin > 0:
        result += 1
    if plusmin == 0:
        result += 0.5
    
    gaa = own_total['ga'] / own_total['icetime'] - opp_total['ga'] / opp_total['icetime']
    if gaa < 0:
        result += 1
    if gaa == 0:
        result += 0.5
    return result

In [943]:
def optimize_lineup(preds, roster, opp_exp):
    lineups = get_valid_lineups(preds[preds.playerId.isin(roster)])        

    if len(lineups) > 0:
        lineup_teams = get_lineup_teams(lineups, preds)
        
        result = lineup_teams.groupby('lineup')[metrics].sum().apply(lambda x: get_expected_points(x, opp_exp), axis=1)
        own_selected_lineup = lineups[result.idxmax()]
        return own_selected_lineup
        
    else:
        return []
    

def get_pct_lineups(day_teams, team_id):
    lineups = get_valid_lineups(day_teams[day_teams.team_id == team_id])
    if len(lineups) == 0:
        return [()]
    lineup_teams = get_lineup_teams(lineups, day_teams)
    metric_sums = lineup_teams.groupby('lineup')[metrics].sum()
    best = metric_sums.rank(pct=True).sum(1).sort_values().index.tolist()
    return [lineups[i] for i in best]

In [944]:
def get_positions_quota(selected_team, pos_lookup):
    positions_quota = {k: v for k, v in POSITIONS_QUOTA.items()}
    lineup_check_players = []
    for player in selected_team:
        if len(pos_lookup[player]) == 1:
            positions_quota[pos_lookup[player][0]] -= 1
        else:
            lineup_check_players.append(player)
    return positions_quota, lineup_check_players

In [945]:
players = players[~players.playerId.duplicated()]
player_info = players.set_index('playerId')[['name','pos']]

In [1085]:
current_team_id = '427.l.21834.t.3'
team_matchups = [m for m in all_matchups if current_team_id in [t.team_key for t in m.teams]]
position_lookup = players.set_index(['playerId'], drop=False).pos.apply(literal_eval).to_dict()

metrics = [m for metrics in PRED_COLS.values() for m in metrics]



selected_team = []
for m in team_matchups[2:]:
    print(m.week)
    opponent_id = [t.team_key for t in m.teams if t.team_key != current_team_id][0]
    dates = pd.date_range(m.week_start, m.week_end)

    week_teams = teams.loc[(pd.to_datetime(teams.index) >= m.week_start)&(pd.to_datetime(teams.index) <= m.week_end)]
    starting_teams = teams.loc[(pd.to_datetime(teams.index) == m.week_start)]
    
    
    week_preds = daily_preds[(daily_preds.date >= m.week_start)&(daily_preds.date <= m.week_end)]
    week_preds = week_preds.merge(week_teams, on=['player_key', 'date'], how='left')
    week_results = daily_results[(daily_results.date >= m.week_start)&(daily_results.date <= m.week_end)]
    
    opp_points = pd.Series({m:0 for metrics in PRED_COLS.values() for m in metrics})
    for date in dates:
        day_teams = week_preds.loc[week_preds.date == date]
        opp_selected_lineup = get_pct_lineups(day_teams, opponent_id)[0]
        opp_selected_team = week_results.loc[week_results.playerId.isin(opp_selected_lineup)&(week_results.date == date)]
        opp_points += get_resulted_points(opp_selected_team)
        
    current_lineup = teams[(teams.team_id == current_team_id)&(teams.index.get_level_values('date') == date)]
    current_lineup = current_lineup.merge(players, how='left', on='player_key').playerId.tolist()

    all_available_players = players[(~players.player_key.isin(starting_teams.player_key))|(players.player_key.isin(current_lineup))]
    all_available_players = all_available_players.playerId.tolist()
    
    if len(selected_team) > 0:
        current_lineup = [p for p in selected_team]
    
    own_points = pd.Series({m:0 for metrics in PRED_COLS.values() for m in metrics})
    
    # a = daily_results[daily_results.date < (m.week_start)].groupby('playerId')[metrics].sum(min_count=1)
    a = week_preds.groupby('playerId')[metrics].mean()
    a['ga'] = -a['ga'] / a['icetime']
    a = a.drop('icetime', axis=1)
    ranks = ((a - a.mean())/(a.std())).mean(1).rank(pct=True)
    top_tier = ranks[ranks > 0.85].index
    
    selected_team = [p for p in current_lineup if p in top_tier]
    while len(selected_team) < 17:
        if len(selected_team) < 15:
            available = [p for p in current_lineup if p not in selected_team]
            
        else:
            available = [p for p in all_available_players if p not in selected_team]
            
        if len(selected_team) > 14:
            week_team_to_check = []

            for date in dates:
                day_sel_team = week_preds[(week_preds.date == date)&(week_preds.playerId.isin(selected_team))].playerId.tolist()
                positions_quota, lineup_check_players = get_positions_quota(day_sel_team, position_lookup)

                players_to_check = []
                for player in available:
                    if check_valid_lineup(lineup_check_players + [player], position_lookup, positions_quota):
                        players_to_check.append(player)
                week_team_to_check.append(week_preds[week_preds.playerId.isin(players_to_check)&(week_preds.date == date)])

            week_team_to_check = pd.concat(week_team_to_check).groupby('playerId')[metrics].sum()
            week_team_to_check += own_points
            test_points = opp_points * (len(selected_team) + 1)

            if len(week_team_to_check) > 0:
                expected_by_p = week_team_to_check.apply(lambda x: get_expected_points(x, test_points), axis=1)
                selected_player = expected_by_p.idxmax()
                own_points = week_team_to_check.loc[selected_player] 
            else:
                selected_player = week_preds.loc[week_preds.playerId.isin(available)].groupby('playerId')[metrics].sum().rank(pct=True).sum(1).idxmax()
        else:
            selected_player = ranks.loc[[p for p in available if p in ranks]].idxmax()
        selected_team.append(selected_player)
        
    print(get_expected_points(own_points, opp_points))
    
    own_result = pd.Series({m:0 for metrics in PRED_COLS.values() for m in metrics})
    opp_result = pd.Series({m:0 for metrics in PRED_COLS.values() for m in metrics})
    for date in dates:
        day_teams = week_preds.loc[week_preds.date == date]
        selected_lineup = optimize_lineup(day_teams, selected_team, opp_points)
        selected_team_res = week_results.loc[week_results.playerId.isin(selected_lineup)&(week_results.date == date)]
        
        opp_selected_lineup = get_pct_lineups(day_teams, opponent_id)[0]
        opp_selected_team_res = week_results.loc[week_results.playerId.isin(opp_selected_lineup)&(week_results.date == date)]
     
        own_result += get_resulted_points(selected_team_res)
        opp_result += get_resulted_points(opp_selected_team_res)
        
    final = get_week_result(own_result, opp_result)
    print(final)
    print(player_info.loc[selected_team])
    print('DROPS')
    for p in current_lineup:
        if p not in selected_team:
            print(player_info.loc[p].to_dict())
    print('ADDS')
    for p in selected_team:
        if p not in current_lineup:
            print(player_info.loc[p].to_dict())


3
1.9282455838374772
6.5
                      name           pos
playerId                                
8477934     Leon Draisaitl   ['C', 'LW']
8478483    Mitchell Marner        ['RW']
8476468        J.T. Miller   ['C', 'RW']
8470794       Joe Pavelski   ['C', 'RW']
8474564     Steven Stamkos   ['C', 'LW']
8479325     Charlie McAvoy         ['D']
8475167      Victor Hedman         ['D']
8473994         Jamie Benn        ['LW']
8477346   MacKenzie Weegar         ['D']
8475753       Justin Faulk         ['D']
8477496     Elias Lindholm         ['C']
8471685       Anze Kopitar         ['C']
8476854    Hampus Lindholm         ['D']
8475184      Chris Kreider        ['LW']
8478492      Ilya Samsonov         ['G']
8478498       Jake DeBrusk  ['LW', 'RW']
8481519     Spencer Knight         ['G']
DROPS
{'name': 'Jacob Markstrom', 'pos': "['G']"}
{'name': 'Tristan Jarry', 'pos': "['G']"}
ADDS
{'name': 'Jake DeBrusk', 'pos': "['LW', 'RW']"}
{'name': 'Spencer Knight', 'pos': "['G']"}
4
1.4868

/var/tmp/ipykernel_2796/1350226997.py:106: RuntimeWarning: invalid value encountered in scalar divide
  gaa = own_total['ga'] / own_total['icetime'] - opp_total['ga'] / opp_total['icetime']


6
                        name           pos
playerId                                  
8477934       Leon Draisaitl   ['C', 'LW']
8478483      Mitchell Marner        ['RW']
8476468          J.T. Miller   ['C', 'RW']
8470794         Joe Pavelski   ['C', 'RW']
8474564       Steven Stamkos   ['C', 'LW']
8479325       Charlie McAvoy         ['D']
8475167        Victor Hedman         ['D']
8473994           Jamie Benn        ['LW']
8477346     MacKenzie Weegar         ['D']
8475753         Justin Faulk         ['D']
8477496       Elias Lindholm         ['C']
8471685         Anze Kopitar         ['C']
8476854      Hampus Lindholm         ['D']
8475184        Chris Kreider        ['LW']
8478498         Jake DeBrusk  ['LW', 'RW']
8477520   Jean-Sebastien Dea         ['C']
8480925       Pavel Francouz         ['G']
DROPS
{'name': 'Ilya Samsonov', 'pos': "['G']"}
{'name': 'Spencer Knight', 'pos': "['G']"}
ADDS
{'name': 'Jean-Sebastien Dea', 'pos': "['C']"}
{'name': 'Pavel Francouz', 'pos': "['G

/var/tmp/ipykernel_2796/1350226997.py:106: RuntimeWarning: invalid value encountered in scalar divide
  gaa = own_total['ga'] / own_total['icetime'] - opp_total['ga'] / opp_total['icetime']


4.5
                      name           pos
playerId                                
8477934     Leon Draisaitl   ['C', 'LW']
8478483    Mitchell Marner        ['RW']
8476468        J.T. Miller   ['C', 'RW']
8470794       Joe Pavelski   ['C', 'RW']
8474564     Steven Stamkos   ['C', 'LW']
8479325     Charlie McAvoy         ['D']
8475167      Victor Hedman         ['D']
8473994         Jamie Benn        ['LW']
8477346   MacKenzie Weegar         ['D']
8475753       Justin Faulk         ['D']
8477496     Elias Lindholm         ['C']
8471685       Anze Kopitar         ['C']
8476854    Hampus Lindholm         ['D']
8475184      Chris Kreider        ['LW']
8478498       Jake DeBrusk  ['LW', 'RW']
8475218     Mattias Ekholm         ['D']
8481519     Spencer Knight         ['G']
DROPS
{'name': 'Jean-Sebastien Dea', 'pos': "['C']"}
{'name': 'Pavel Francouz', 'pos': "['G']"}
ADDS
{'name': 'Mattias Ekholm', 'pos': "['D']"}
{'name': 'Spencer Knight', 'pos': "['G']"}
6
1.992518443857031
6
        

...

/var/tmp/ipykernel_2796/1350226997.py:106: RuntimeWarning: invalid value encountered in scalar divide
  gaa = own_total['ga'] / own_total['icetime'] - opp_total['ga'] / opp_total['icetime']


4.5
                      name           pos
playerId                                
8477934     Leon Draisaitl   ['C', 'LW']
8478483    Mitchell Marner        ['RW']
8476468        J.T. Miller   ['C', 'RW']
8470794       Joe Pavelski   ['C', 'RW']
8474564     Steven Stamkos   ['C', 'LW']
8479325     Charlie McAvoy         ['D']
8475167      Victor Hedman         ['D']
8477346   MacKenzie Weegar         ['D']
8477496     Elias Lindholm         ['C']
8471685       Anze Kopitar         ['C']
8478042   Viktor Arvidsson  ['LW', 'RW']
8470613        Brent Burns         ['D']
8480039       Martin Necas   ['C', 'RW']
8476931        Jake McCabe         ['D']
8475753       Justin Faulk         ['D']
8478449        Roope Hintz         ['C']
8471306      Thomas Greiss         ['G']
DROPS
{'name': 'Jesper Wallstedt', 'pos': "['G']"}
{'name': 'Eric Staal', 'pos': "['C']"}
ADDS
{'name': 'Roope Hintz', 'pos': "['C']"}
{'name': 'Thomas Greiss', 'pos': "['G']"}
25
0.9921471169081922
4.5
              

/var/tmp/ipykernel_2796/1350226997.py:106: RuntimeWarning: invalid value encountered in scalar divide
  gaa = own_total['ga'] / own_total['icetime'] - opp_total['ga'] / opp_total['icetime']


1.4406925185192005
6
                       name           pos
playerId                                 
8477934      Leon Draisaitl   ['C', 'LW']
8478483     Mitchell Marner        ['RW']
8476468         J.T. Miller   ['C', 'RW']
8470794        Joe Pavelski   ['C', 'RW']
8474564      Steven Stamkos   ['C', 'LW']
8479325      Charlie McAvoy         ['D']
8475167       Victor Hedman         ['D']
8477346    MacKenzie Weegar         ['D']
8477496      Elias Lindholm         ['C']
8471685        Anze Kopitar         ['C']
8478042    Viktor Arvidsson  ['LW', 'RW']
8480039        Martin Necas   ['C', 'RW']
8476931         Jake McCabe         ['D']
8475753        Justin Faulk         ['D']
8477501   Valeri Nichushkin  ['LW', 'RW']
8481043         Cole Koepke        ['LW']
8477293        Antti Raanta         ['G']
DROPS
{'name': 'Brent Burns', 'pos': "['D']"}
{'name': 'Carter Hart', 'pos': "['G']"}
ADDS
{'name': 'Cole Koepke', 'pos': "['LW']"}
{'name': 'Antti Raanta', 'pos': "['G']"}


/var/tmp/ipykernel_2796/1350226997.py:106: RuntimeWarning: invalid value encountered in scalar divide
  gaa = own_total['ga'] / own_total['icetime'] - opp_total['ga'] / opp_total['icetime']


,g,a,sog,fow,hit,block,pim,goalsfor,goalsaga,ppp,ga,win,so,save,icetime
playerId,,,,,,,,,,,,,,,
8467950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.050849,0.449708,2.036116,22.638546,9.101022
8470594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.850267,0.589323,1.774750,29.296628,12.413836
8470595,0.550899,0.783327,4.406944,12.099042,1.349061,1.826158,0.698274,1.550866,1.305330,0.033238,NaN,NaN,NaN,NaN,NaN
8470600,0.166634,0.709077,4.337617,0.077084,2.548658,3.999487,1.027794,2.399506,1.998164,0.053800,NaN,NaN,NaN,NaN,NaN
8470604,0.418855,0.463803,4.070273,10.474483,2.994903,1.524301,1.059521,0.886485,1.054368,0.175631,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8484304,0.000000,0.042615,0.639940,0.062302,0.589641,0.843156,0.332463,1.276947,0.720590,0.004526,NaN,NaN,NaN,NaN,NaN
8484312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.030549,0.032049,2.761376,8.179810,2.263505
8484314,0.045003,0.042615,0.639940,0.062302,0.723000,0.121137,0.287359,1.337642,1.063036,0.004526,NaN,NaN,NaN,NaN,NaN


In [1077]:
player_info.loc[[p for p in current_lineup if p in top_tier]]

,name,pos
playerId,,
8477934,Leon Draisaitl,"['C', 'LW']"
8476468,J.T. Miller,"['C', 'RW']"
8474564,Steven Stamkos,"['C', 'LW']"
8475167,Victor Hedman,['D']
8471685,Anze Kopitar,['C']
8478483,Mitchell Marner,['RW']
8470794,Joe Pavelski,"['C', 'RW']"
8477346,MacKenzie Weegar,['D']
8477496,Elias Lindholm,['C']


In [ ]:
]